<a href="https://colab.research.google.com/github/bokutachi256/gisday2019/blob/master/1_%E5%9F%BA%E7%9B%A4%E5%9C%B0%E5%9B%B3%E6%83%85%E5%A0%B1%E3%81%AE%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pysheds

ここでは基盤地図情報の数値標高モデル（DEM: Digital Elevation Model）を用いて分析する方法について解説していきます．

まず，[基盤地図情報のwebサイト](http://www.gsi.go.jp/kiban/)からDEMをダウンロードします．
ダウンロードしたデータは`FG-GML-5139-03-50-DEM5A-20161001.xml`とします．

このデータはXML型式になっていますので，
Pythonでxmlを扱うために`xml.etree.ElementTree`を`ET`として読み込みます．


In [0]:
import xml.etree.ElementTree as ET
#import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#import matplotlib.colors as colors
#import mathmath
#import glob
import pandas as pd
import io
#from osgeo import gdal, gdalconst, gdal_array
#from pysheds.grid import Grid

from google.colab import drive

In [0]:
drive.mount('/content/drive')
base_dir = "/content/drive/My Drive/gisday2019/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


基盤地図情報のxmlを読み込んでparseします．
オブジェクト`tree`のルート要素を取得し，`elem`に代入します．

In [0]:
tree = ET.parse(base_dir + "GSI-DEM/FG-GML-5139-03-50-DEM5A-20161001.xml")
elem = tree.getroot()

`elem`を確認してみます．

In [0]:
elem

<Element '{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}Dataset' at 0x7fd562804408>

ルート要素のタグ`Dataset`が表示されました．

次にイテレータを使ってすべてのタグを取得し，それを表示してみます．


In [0]:
for e in elem.getiterator():
    print(e.tag)

{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}Dataset
{http://www.opengis.net/gml/3.2}description
{http://www.opengis.net/gml/3.2}name
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}DEM
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}fid
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}lfSpanFr
{http://www.opengis.net/gml/3.2}timePosition
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}devDate
{http://www.opengis.net/gml/3.2}timePosition
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}orgGILvl
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}orgMDId
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}type
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}mesh
{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}coverage
{http://www.opengis.net/gml/3.2}boundedBy
{http://www.opengis.net/gml/3.2}Envelope
{http://www.opengis.net/gml/3.2}lowerCorner
{http://www.opengis.net/gml/3.2}upperCorner
{http://www.opengis.net/gml/3.2}gridDomain
{http://www.opengis.net/gml/3.2}Grid
{http://www.opengis.net/gml/3.2}limits
{http://ww

基盤地図情報DEMのXMLは名前空間付きXMLです．
このため，タグへのアクセスには名前空間の接頭辞をつける必要があります．
たとえば，`find`メソッドを使ってタグ`mesh`にアクセスする場合，
通常のXMLのようにアクセスするとエラーが発生します．

In [0]:
mesh = elem.find(".//mesh")
print(mesh.text)

AttributeError: ignored


これを解決するためには，タグに紐づけられている名前空間のURIをタグごとに加える必要があります．
例えば以下のようにです．


In [0]:
mesh = elem.find(".//{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}mesh")
print(mesh.text)

51390350


しかし，すべてのタグにURIをつけるのは煩雑です．
プログラミングミスの原因にもなりかねません．
もう少しエレガントな方法としては，
タグのディクショナリをメソッドの第2引数に指定する方法があります．

以下はDEMのXMLの冒頭部です．
xmlnsで始まる部分が名前空間の定義になっており，それぞれURIが指定されています．


```xml
<?xml version="1.0" encoding="UTF-8"?>
<Dataset
  gml:id="Dataset1"
  xmlns="http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema" 
  xmlns:xlink="http://www.w3.org/1999/xlink"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:gml="http://www.opengis.net/gml/3.2"
  xsi:schemaLocation="http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema FGD_GMLSchema.xsd"
>
```

これを元にして名前空間の接頭辞のディクショナリを作成します．
`xmlns="http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema"`
はデフォルトの名前空間を指定しています．


In [0]:
ns = {'default': 'http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema',
  'xlink': 'http://www.w3.org/1999/xlink',
  'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'gml': 'http://www.opengis.net/gml/3.2'}

```xml
<?xml version="1.0" encoding="UTF-8"?>
<Dataset
  gml:id="Dataset1"
  xmlns="http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema" 
  xmlns:xlink="http://www.w3.org/1999/xlink"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:gml="http://www.opengis.net/gml/3.2"
  xsi:schemaLocation="http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema FGD_GMLSchema.xsd"
>
  <gml:description>基盤地図情報メタデータ ID=fmdid:15-3101</gml:description>
  <gml:name>基盤地図情報ダウンロードデータ（GML版）</gml:name>
  <DEM gml:id="DEM001">
    <fid>fgoid:10-00100-15-60101-51390350</fid>
    <lfSpanFr gml:id="DEM001-1">
      <gml:timePosition>2016-10-01</gml:timePosition>
    </lfSpanFr>
    <devDate gml:id="DEM001-2">
      <gml:timePosition>2016-10-01</gml:timePosition>
    </devDate>
    <orgGILvl>0</orgGILvl>
    <orgMDId>H24GC009 NoData</orgMDId>
    <type>5mメッシュ（標高）</type>
    <mesh>51390350</mesh>
    <coverage gml:id="DEM001-3">
      <gml:boundedBy>
        <gml:Envelope srsName="fguuid:jgd2011.bl">
          <gml:lowerCorner>34.041666667 139.375</gml:lowerCorner>
          <gml:upperCorner>34.05 139.3875</gml:upperCorner>
        </gml:Envelope>
      </gml:boundedBy>
      <gml:gridDomain>
        <gml:Grid gml:id="DEM001-4" dimension="2">
          <gml:limits>
            <gml:GridEnvelope>
              <gml:low>0 0</gml:low>
              <gml:high>224 149</gml:high>
            </gml:GridEnvelope>
            </gml:limits>
            <gml:axisLabels>x y</gml:axisLabels>
        </gml:Grid>
      </gml:gridDomain>
      <gml:rangeSet>
        <gml:DataBlock>
          <gml:rangeParameters>
            <gml:QuantityList uom="DEM構成点"/>
          </gml:rangeParameters>
          <gml:tupleList>地表面,1.21 地表面,1.49. .......
        </gml:DataBlock>
      </gml:rangeSet>
      <gml:coverageFunction>
        <gml:GridFunction>
          <gml:sequenceRule order="+x-y">Linear</gml:sequenceRule>
          <gml:startPoint>155 12</gml:startPoint>
        </gml:GridFunction>
      </gml:coverageFunction>
    </coverage>
  </DEM>
</Dataset>
```

`elem`オブジェクトの`find`メソッドにURI付きのタグ`{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}mesh`を入力してタグを検索し，
その値を表示してみましょう．
戻り値（このタグの値）は標準地域メッシュ番号となっています．
最初の4桁`5139`が1次メッシュコード，
次の2桁が`03`が2次メッシュコード，
最後の2桁`50`が3次メッシュコードとなっています．

In [0]:
elem.find(".//{http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema}mesh").text

'51390350'

In [0]:
mesh = elem.find('.//default:mesh', ns)
print(mesh.text)

51390350


In [0]:
id_board = elem.find('.//gml:startPoint', ns) 
print(id_board.text) # 54321

155 12


次に`{http://www.opengis.net/gml/3.2}lowerCorner`を表示してみます．
このタグにはDEMの南東端の座標が格納されています．

In [0]:
elem.find('.//gml:lowerCorner', ns).text

'34.041666667 139.375'

同様に`{http://www.opengis.net/gml/3.2}upperCorner`には
北西端の座標が格納されています．

In [0]:
elem.find('.//gml:upperCorner', ns).text

'34.05 139.3875'

`{http://www.opengis.net/gml/3.2}high`には
3次メッシュ内の標高値の数が格納されています．
5mメッシュでは224$\times$149メッシュになります．

In [0]:
elem.find(".//{http://www.opengis.net/gml/3.2}high").text

'224 149'

タグ`{http://www.opengis.net/gml/3.2}tupleList`を検索し，値を表示する．
ここに標高が格納されている．
標高は文字列として格納されている．
改行コードとカンマで分けられている．
カンマの前が標高点の属性，カンマの後ろが標高値になっている．
データがないメッシュの属性は'データなし'で標高値は'-9999'，
データがあるメッシュの属性は'地表面'で標高値が実数で入っている．

In [0]:
elev = elem.find(".//{http://www.opengis.net/gml/3.2}tupleList").text
print(elev)

標高値が格納されている文字列はcsvそのものになっており，
csvとして書き出せばpandasのpd.read_csv()でデータフレームに読み込むことができる．

しかし，いちいちcsvに書き出すのも面倒なため，io.StringIOを使ってテキストストリームを作成し，これにpandasのpd.read.csvringIOを組み合わせて
csv型式の文字列をデータフレームに格納する．

In [0]:
df = pd.read_csv(io.StringIO(elev), header=None)
df

,0,1
0,地表面,1.21
1,地表面,1.49
2,地表面,1.18
3,データなし,-9999.00
4,データなし,-9999.00
...,...,...
21160,データなし,-9999.00
21161,データなし,-9999.00
21162,データなし,-9999.00
21163,データなし,-9999.00


朝日航洋レーザーデータから作成したDEMファイルを
ArcGISで読み込めるASCIIラスタ形式に変換するプログラム

In [0]:
import pandas as pd
import glob

処理対象のファイル名を取得する．
処理対象は*.txtとする

In [0]:
infiles = glob.glob("*.txt")

In [0]:
for infile in infiles:

#データフレーム`dem`に標高データを読み込む．

#その後，列名に`X`，`Y`，`Z`をつける.

    dem = pd.read_csv(infile, delim_whitespace=True, header=-1)
    dem.columns = ['X', 'Y', 'Z']
    

#NoData（1.701410e+38）の値を-32768に置き換える．

    dem['Z2'] = dem['Z'].where(dem['Z'] != 1.701410e+38, -32768)

#縦持ちの標高データを横持ちに変換する．
#その後，Y方向でソートして南北の順を整列させる．

    asc_ras = dem.pivot_table(index = 'Y', columns = 'X', values = 'Z2', fill_value = 0)
    asc_ras = asc_ras.sort_index(ascending = False)

#ASCIIラスターのヘッダをつける．
#ヘッダの型式は以下になっている.

#```
#ncols 2000
#nrows 1500
#xllccenter 378923
#yllcenter 4072345
#cellsize 1
#nodata_value -32768
#43 2 45 7 3 56 2 5 23 65 34 6 32 54 57 34 2 2 54 6 
#35 45 65 34 2 6 78 4 2 6 89 3 2 7 45 23 5 8 4 1 62 ...
#```

#ヘッダは横持ちにしたDEMから取得する.

    headers = "ncols %d\n" % len(asc_ras.columns) \
        + "nrows %d\n" % len(asc_ras.index) \
        + "xllcenter %f\n" % dem['X'].min() \
        + "yllcenter %f\n" % dem['Y'].min() \
        + "cellsize 1\n" \
        + "nodata_value -32768\n"

#ヘッダを書き出す

    f = open(infile+'.asc', 'w')
    f.write(headers)

#横持ちにしたDEM本体をアペンドモードで書き出す．

    with open(infile+'.asc', 'a') as f:
        asc_ras.to_csv(f, index = False, header = False, sep =' ')

基盤地図情報DEM（XML型式）を読み込んでGeoTiffにする．

In [0]:
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from osgeo import gdal
from google.colab import drive
from tqdm import tqdm
import xml.etree.ElementTree as ET

In [0]:
drive.mount('/content/drive')
base_dir = '/content/drive/My Drive/workspace/GDAL_Test/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


XMLを読み込み，rootを取得

In [0]:
tree = ET.parse(base_dir + 'FG-GML-4930-17-70-DEM5A-20161001.xml')
root = tree.getroot()


In [0]:
root.attrib

{'{http://www.opengis.net/gml/3.2}id': 'Dataset1',
 '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://fgd.gsi.go.jp/spec/2008/FGD_GMLSchema FGD_GMLSchema.xsd'}

In [0]:
tree = ET.parse(base_dir + 'FG-GML-4930-17-70-DEM5A-20161001.xml')
print(tree.tag, tree.attrib)

AttributeError: ignored

In [0]:
print(tree)

In [0]:
with urllib.request.urlopen(req) as response:
    xml_string = response.read()
 
root = ET.fromstring(xml_string)